# How to read data from DataFrames with pyActigraphy

**Original Author:** Grégory Hammad

**Note:** *This is a legacy tutorial originally developed for `pyActigraphy`. It remains useful for reference, but will be updated following the release of the first milestone version of the `circStudio` package.*

**Observation:** *Once the first milestone version of the `circStudio` package is released, this will become the default method for opening files. Input files must be pre-formatted to ensure they can be properly cleaned and analyzed.*

## Imported packages and input data

The usual suspects:

In [1]:
import numpy as np

In [2]:
import os

In [3]:
import pandas as pd

In [4]:
import circStudio

In this example, let's generate some input data:

NB: if you already have your data under a pandas.DataFrame format, jump directly to the next section.

In [5]:
N = 1440*7 # 7 days of acquisition at a frequency of 60s.

In [6]:
activity = np.random.normal(10,1,N)
light = np.random.normal(100,10,N)

In [7]:
non_wear = np.empty(N)

In [8]:
# Set up a segment of  spurious inactivity
activity[2060:2160] = 0.0
non_wear[2060:2160] = 1.0

In [9]:
d = {'Activity': activity, 'Light': light, 'Non-wear': non_wear}

In [10]:
index = pd.date_range(start='01-01-2020',freq='60s',periods=N)

In [11]:
data = pd.DataFrame(index=index,data=d)

In [12]:
data

,Activity,Light,Non-wear
2020-01-01 00:00:00,11.330084,110.264988,8.517301e-312
2020-01-01 00:01:00,9.930034,95.676325,8.512474e-312
2020-01-01 00:02:00,8.159806,81.785469,0.000000e+00
2020-01-01 00:03:00,10.627438,94.571044,0.000000e+00
2020-01-01 00:04:00,11.026022,89.978322,0.000000e+00
...,...,...,...
2020-01-07 23:55:00,8.365821,98.970412,0.000000e+00
2020-01-07 23:56:00,11.611638,114.083911,0.000000e+00
2020-01-07 23:57:00,11.024646,107.674498,0.000000e+00
2020-01-07 23:58:00,10.507091,111.895947,0.000000e+00


## Manual creation of a BaseRaw object

In [13]:
from circStudio.io import Raw

### Set activity and light data (if available)

In [14]:
raw = Raw(
    start_time=data.index[0],
    period=(data.index[-1]-data.index[0]),
    frequency=data.index.freq,
    activity=data['Activity'],
    light=data['Light']
)

In [15]:
raw.activity

2020-01-01 00:00:00     8.041281
2020-01-01 00:01:00     8.491735
2020-01-01 00:02:00    10.507292
2020-01-01 00:03:00     9.716230
2020-01-01 00:04:00    10.716860
                         ...    
2020-01-07 23:55:00     9.146460
2020-01-07 23:56:00    10.896237
2020-01-07 23:57:00    11.078052
2020-01-07 23:58:00     9.443088
2020-01-07 23:59:00    12.226668
Freq: 60s, Name: Activity, Length: 10080, dtype: float64

### Opening a file not natively supported by pyActigraphy

In [6]:
# Open the tsv (tab separated file) file
df = pd.read_csv(os.path.join('esteban.txt'), sep='\t')

# Create a datatime column, containing timestamps, and drop the individual 'date' and 'time' columns
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='%d/%m/%y %H:%M:%S')
df = df.drop(['date', 'time'], axis=1)

# Set the datetime column as an index
df = df.set_index('datetime')

# Set the frequency (in this case, 30 seconds)
df.index.freq = '30s'

In [7]:
# Create a new BaseRaw object
raw = Raw(
    name="estevan", 
    uuid='kw3',
    axial_mode=None,
    start_time=df.index[0],
    period=(df.index[-1]-df.index[0]),
    #frequency=pd.Timedelta(30,'s'),
    frequency = df.index.freq,
    data=df['t_mov'], # Assuming that activity is stored here
    light=df['visible_light'] # Working with visible_light
)

### Set up a mask

Most devices that have a wear sensor return this information as a binary time series with "1" when the device is most likely not worn and "0" otherwise.
In pyActigraphy, this information can be used to create a mask and thus invalidate the corresponding data points (set to "0" most probably). However, the mask, the value "1" correspond to "no masking". So, depending on your "non-wear" data, be careful to transform them appropriately:

In [16]:
# Here, I assume that 0: the device is worn, 1: device not worn. 
# As mentioned aboce, for the mask, 1: no masking. (NB: it is a convolution: data*mask)
raw.mask = np.abs(data['Non-wear']-1)

## Tests

In [17]:
raw.duration()

<604800 * Seconds>

In [18]:
raw.length()

10080

In [19]:
raw.ADAT(binarize=False)

AttributeError: 'BaseRaw' object has no attribute 'data'

In [20]:
raw.IV(binarize=False)

AttributeError: 'BaseRaw' object has no attribute 'resampled_data'

In [21]:
# If you want to mask the data
raw._mask_inactivity = True

In [22]:
# For a gaussian noise, IV should be close to 2.
raw.IV(binarize=False)

1.9908499316790234

The masking seems to work!

Et voilà!